# Initializations

In [ ]:
!pip install PyPDF2
!pip install bertopic
!pip install top2vec
!pip install top2vec[sentence_transformers]
!pip install top2vec[indexing]
!pip install top2vec[sentence_encoders]
!sudo apt-get install poppler-utils
!pip install top2vec[sentence_transformers]

import shutil
from google.colab import files

import PyPDF2
from top2vec import Top2Vec

from PyPDF2 import PdfReader, PdfWriter, PdfMerger, PdfFileMerger
from glob import glob
import re
from pathlib import Path
import os

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download("popular")
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer

import random

import pandas as pd
import numpy as np
import multiprocessing
import time

multiprocessing.cpu_count()


#PDF to Text conversion

pdf_convertor pdf to text convertor function that converts a single pdf file to text

In [ ]:
"""

  pdf_convertor is a function that takes 5 arguments. It converts a pdf file to a text file,
  and (optionally) trims the header and footer
  Parameters:

      pdf_path(string) = file path of pdf to be converted
  
      new_path(string) = file path of the text file that is created 

      lowerbound(int) = user inputs lower bound co-ordinates for footer cropping
      
      upperbound(int) = user input upper bound co-ordinates for header cropping

      trimheaders(bool) = user inputs true if they want the header and footer trimmed, false otherwise

      dropfirst(bool) = user inputs true if they want the first page cut out, false otherwise

  Returns:
      
      none


"""

#function should have arguments added for dimensions of trimmed f/h down the road

def pdf_convertor(pdf_path, new_path, lowerbound, upperbound, trimheaders, dropfirst):    
    bounds = 0
    if dropfirst:
      bounds = 1


    reader = PdfReader(pdf_path, 'rb')

    parts = []
    text_body = ""


    num_words_before = 0
    for page_num in range(len(reader.pages)):
      page = reader.pages[page_num]
      num_words_before += len(page.extract_text().split())
    print(f"Number of words before conversion: {num_words_before}")
    
    
    #for signs 1979 u want y > 70 and y < 600

    def visitor_body(text, cm, tm, fontDict, fontSize):
      y = tm[5]
      if y > lowerbound and y < upperbound:
        parts.append(text)

    for current_page in range(bounds,len(reader.pages)):
      page = reader.pages[current_page]
      if trimheaders:
        page.extract_text(visitor_text=visitor_body)
        text_body = "".join(parts)
      elif trimheaders is False:
        text = page.extract_text()
        parts.append(text)
        text_body = "".join(parts)
      else:
        print("please input true if you would like header/footer trimmed, else input false")
        return

    # Count number of words after conversion
    num_words_after = len(text_body.split())
    print(f"Number of words after conversion: {num_words_after}")

    
    newfile = open(new_path, "w") 
    newfile.write(text_body)
    newfile.close()
    #print(text_body)

convert_pdfs_to_txts converts to text all pdf files in a directory. It calls the pdf_convertor function to do the actual conversion. It then saves converted files in a mirrored directory.

In [ ]:
def convert_pdfs_to_txts(root_path, output_path, journal_params):
    """
    Converts all the pdf files present in the given root directory to txt files and saves them in the
    output directory in a folder structure identical to the root directory.

    Parameters:
    root_path (str): The path to the root directory containing pdf files to be converted.
    output_path (str): The path to the output directory where the converted txt files will be saved.
    journal_params (dict): A dictionary containing parameters for each journal.

    Returns:
    None
    """
    print('=========================================')
    print('Converting given pdfs to txts...')
    # Iterate over each journal
    for journal in journal_params.keys():
        print('=========================================')
        print(f"Converting journal {journal}...")
        # Iterate over each year folder for the current journal
        for year_folder in os.listdir(os.path.join(root_path, journal)):
            print(f"\n\nYear {year_folder}...")
            # Iterate over each pdf file in the current year folder
            for i, pdf_file in enumerate(os.listdir(os.path.join(root_path, journal, year_folder))):
                
                # Extract the year from the year folder name
                year = year_folder

                # Create the output folder path
                output_folder_path = os.path.join(output_path, journal, year)

                # Create the output folder if it does not exist
                if not os.path.exists(output_folder_path):
                    os.makedirs(output_folder_path)

                # Create the input and output file paths
                input_file_path = os.path.join(root_path, journal, year_folder, pdf_file)
                output_file_path = os.path.join(output_folder_path, "{}_{}_{}.txt".format(journal, year, i+1))

                # Extract parameters for the current journal
                journal_params_dict = journal_params[journal][year]
                lowerbound = journal_params_dict.get('lowerbound', 0)
                upperbound = journal_params_dict.get('upperbound', 1000)
                trimheaders = journal_params_dict.get('trimheaders', False)
                dropfirst = journal_params_dict.get('dropfirst', False)

                print(f'path_to_pdf {input_file_path}')
                print(f'path_to_txt {output_file_path}')

                # Convert the current pdf file to txt file
                pdf_convertor(input_file_path, output_file_path, lowerbound, upperbound, trimheaders, dropfirst)

#Pdf to Text Convertor (remove Header and Footer) bash version WIP

---



> this version utilizes the bash library pdftotext. You must cd into the directory where the files are. this is bandaid code for when the convertor function utilizing PyPdf2 doesn't produce the correct results. It is not yet understood why this happens, and so far it seems to be an issue with PyPdf2 and specific pdf formats.  I recommend using the other function when possible. this is wip. it will eventually be a function that will take path names and args. 

In [ ]:
%cd '/content/gdrive/MyDrive/2018fstudiespdfs'
directory = '/content/gdrive/MyDrive/2018fstudiespdfs/'
files = Path(directory).glob('*.pdf')

i = 1

for file in files:
  p = os.path.basename(file)
  pdf_name = p + " "
  text_name = str(i)+ "results.txt"
  bash_command = "pdftotext -x 0 -y 50 -W 794 -H 590 " + pdf_name + text_name 

  # uncomment which one you need. there needs to be a way to make this into a function. 

  #text = re.sub("\sAcknowledgments((.|\n)*)"," ", text) this isnt written right
  #text = re.sub("\nNotes((.|\n)*)((.|\n)*)"," ", text)
  #text = re.sub("\sNotes((.|\n)*)((.|\n)*)"," ", text)
  #text = re.sub("\sReferences((.|\n)*)((.|\n)*)"," ", text)
  #text = re.sub("\sAppendix:((.|\n)*)((.|\n)*)"," ", text)
  #y is top margin, w is width, h is bottom margin 
  # one line is equal to 20 points in coord system (at least for F.Review 2018)

  os.system(bash_command)
  i += 1


# PDF to Text Conversion in Action

In [ ]:
!unzip raw_pdfs.zip

In [ ]:
# Example code with initializations

root_path = "raw_pdfs"
output_path = "text_samples"

# Define journal parameters
journal_params = {
    'Feminist_Review': {
        '1979': {'lowerbound': 75, 'upperbound': 675, 'trimheaders': True, 'dropfirst': True},
        '1989': {'lowerbound': 40, 'upperbound': 595, 'trimheaders': True, 'dropfirst': True},
        '1999': {'lowerbound': 35, 'upperbound': 600, 'trimheaders': True, 'dropfirst': True},
        '2009': {'lowerbound': 90, 'upperbound': 675, 'trimheaders': True, 'dropfirst': True},
        '2019': {'lowerbound': 75, 'upperbound': 650, 'trimheaders': True, 'dropfirst': True}
    }, 
    'Feminist_Studies': {
        '1979': {'lowerbound': 40, 'upperbound': 575, 'trimheaders': True, 'dropfirst': True},
        '1989': {'lowerbound': 50, 'upperbound': 575, 'trimheaders': True, 'dropfirst': True},
        '1999': {'lowerbound': 30, 'upperbound': 575, 'trimheaders': True, 'dropfirst': True},
        '2009': {'lowerbound': 70, 'upperbound': 575, 'trimheaders': True, 'dropfirst': True},
        '2019': {'lowerbound': 70, 'upperbound': 575, 'trimheaders': True, 'dropfirst': True}
    },
    'Frontiers': {
        '1979': {'lowerbound': 48, 'upperbound': 720, 'trimheaders': True, 'dropfirst': True},
        '1989': {'lowerbound': 48, 'upperbound': 720, 'trimheaders': True, 'dropfirst': True},
        '1999': {'lowerbound': 90, 'upperbound': 590, 'trimheaders': True, 'dropfirst': True},
        '2009': {'lowerbound': 90, 'upperbound': 640, 'trimheaders': True, 'dropfirst': True},
        '2019': {}
    },
    'Signs': {
        '1979': {'lowerbound': 70, 'upperbound': 590, 'trimheaders': True, 'dropfirst': False},
        '1989': {'lowerbound': 80, 'upperbound': 590, 'trimheaders': True, 'dropfirst': False},
        '1999': {},
        '2009': {},
        '2019': {}
    },
    'Women_Studies_Quarterly': {
        '1979': {},
        '1989': {'lowerbound': 70, 'upperbound': 575, 'trimheaders': True, 'dropfirst': True},
        '1999': {'lowerbound': 70, 'upperbound': 600, 'trimheaders': True, 'dropfirst': True},
        '2009': {'lowerbound': 95, 'upperbound': 600, 'trimheaders': True, 'dropfirst': True},
        '2019': {'lowerbound': 95, 'upperbound': 600, 'trimheaders': True, 'dropfirst': True}
    }
}

# Call the convert_pdfs_to_txts function for each journal and year
convert_pdfs_to_txts(root_path, output_path, journal_params)

#Cut Reference Pages

In [ ]:
"""
remove_references cleans the reference pages present in all of the text files 
in the given root directory and saves them in the
output directory in a folder structure identical to the root directory.
Checks for files whose contents have been reduced by 50% or more, and if found, 
prints out a list with their count and information about each one.

Parameters:
source_dir (str): path of source directory
target_dir (str): The path to the output directory where the cleaned text files will be saved

Returns:
None
"""

def remove_references(source_dir, target_dir):
    file_count = 0
    outliers = []
    # Create the target directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Traverse the source directory
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.txt'):
                # Construct the source and target file paths
                source_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, source_dir)
                target_subdir = os.path.join(target_dir, relative_path)
                if not os.path.exists(target_subdir):
                    os.makedirs(target_subdir)
                target_file_path = os.path.join(target_subdir, file)

                # Read the source file
                with open(source_file_path, 'r', encoding='utf-8') as source_file:
                    text = source_file.read()

                # Count words before applying the regex
                word_count_before = len(re.findall(r'\w+', text))

                # Apply the regex
                # Regular expression pattern
                pattern = r"((?:^|\n|\s{2,}))(references|bibliography|selected bibliography|notes|suggestions for further reading|acknowledgements|works cited)(\s*(:\s*|\n))[\s\S]*"

                # Define a custom replace function
                def replace_matched(match):
                    return match.group(1)

                # Remove everything at and after the specified words
                no_ref_text = re.sub(pattern, replace_matched, text, flags=re.IGNORECASE)

                # Count words after applying the regex
                word_count_after = len(re.findall(r'\w+', no_ref_text))

                # Print the word counts
                print(f"File: {source_file_path}")
                print(f"Word count before: {word_count_before}")
                print(f"Word count after: {word_count_after}\n")
                if word_count_after <= 0.5 * word_count_before:
                    new_entry = {'File:  ': source_file_path, 'Word_Count_Before:  ': word_count_before, 'Word_Count_After:  ': word_count_after}
                    outliers.append(new_entry)
                # Save the processed content to the target file
                with open(target_file_path, 'w', encoding='utf-8') as target_file:
                    target_file.write(no_ref_text)
                file_count += 1

    if outliers:
        print('=========================================')
        print('POTENTIAL ERROR WARNING')
        print('There are ' + str(len(outliers)) + ' truncated files out of ' +str(file_count))
        print('here is a list of the files where half or more of text was cut after reference page removal..\n\n')
    

        for item in outliers:
            for key in item:
                print(key,item[key])
            print('-' * 20)
    else:
        print('=========================================')
        print('NO ERRORS DETECTED')

In [ ]:
remove_references("converted_text","no_references")

#Cleaner Function
Cleans the text. Comment out any regex you don't need.

In [ ]:
"""
clean_txt_files cleans all the text files present in the given root directory and saves them in the
output directory in a folder structure identical to the root directory.
Checks for files whose contents have been reduced by 50% or more, and if found, 
prints out a list with their count and information about each one.

Parameters:
source_dir (str): path of source directoru
target_dir (str): The path to the output directory where the cleaned text files will be saved

Returns:
None
"""

def clean_txt_files(source_dir, target_dir):
   
    file_count = 0
    outliers = [
    ]
    # Create the target directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Traverse the source directory
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.txt'):
                # Construct the source and target file paths
                source_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, source_dir)
                target_subdir = os.path.join(target_dir, relative_path)
                if not os.path.exists(target_subdir):
                    os.makedirs(target_subdir)
                target_file_path = os.path.join(target_subdir, file)

                # Read the source file
                with open(source_file_path, 'r', encoding='utf-8') as source_file:
                    content = source_file.read()

                # Count words before applying the regex
                word_count_before = len(re.findall(r'\w+', content))

                # Apply the regex
                content = content.casefold() #makes all content lowercase
                content = re.sub("\shttp\:+.*", " ", content) #removes http till end of line
                content = re.sub("\shttps\:+.*", " ", content) #removes http till end of line
                content = content.replace("\n", " ") #gets rid of newlines
                content = re.sub("\sed\.+\s", " ", content) #gets rid of editor reference
                content = re.sub("\sno+\.", " ", content) #removes no.
                content = re.sub("\sco+\.", " ", content) #removes co.
                content = re.sub("\spp+\.", " ", content) #removes pp.
                content = re.sub("\svol+\.", " ", content) #removes vol.
                content = re.sub("ibid+\.", " ", content) #removes ibid.
                content = re.sub("\sfig\s", " ", content) #removes reference to figures
                content = re.sub("\schiltable\s", " ", content) #removes reference to chiltable
                content = re.sub("\s[a-z]\."," ", content) # all single chars followed by period
                content = re.sub("[a-z]+\.+\:", " ", content) #gets rid of interview abrev. i.e. (o.:)
                content = re.sub("\s+\?+\s+[^.]*\.", " ", content) # gets rid of those funky ? university of chicago statements
                content = re.sub("[ ]+\d+\.+\s+[^.]*\.", " ", content) #gets rid of citations(most of them 0__0)
                content = re.sub("sources+\.+\-+[^.]+\."," ", content) #gets rid of sources:-
                content = re.sub("source+\.+\-+[^.]+\."," ", content)  #gets rid of source:-
                content = re.sub("p{Pd}", " ", content) #attempts at dealing with the emdash
                content = re.sub('\â€”', " ", content) #another attempt at dealing with the emdash
                content = re.sub("\-\s+", "", content) #ensures words that are split mid page are reconstructed
                content = re.sub("\-", " ", content) #compound words are split
                content = re.sub("[^a-z.\"]+\d+[^a-z]*", " ", content) #removes all series of numbers and characters with no words in between them
                content = re.sub("\.+\.+", " ", content) #removes all series of periods
                content = re.sub("etc+\.", " ", content) #removes all etc. (irrelevant and messes with sentence tokenization)
                content = re.sub("\d+"," ", content) #removes remaining numbers
                content = re.sub("\/", " ", content) #seperating words with a space that are seperated by a virgule in the content
                content = re.sub("\s[^a,i]\s", " ", content) #remove all single letters that are not a or i
                content = re.sub("\s[^a,i]\s", " ", content) #repeat due to regex silliness
                content = re.sub("\.+[a-z]\s", ". ", content) # replace all occurences of .(letter) with .
                content = re.sub("\w\.\w+", " ", content) #removes all words with periods inside them
                #content = re.sub(r"\([^()]*\)", " ", content) #gets rid of everything in parentheses
                #content = re.sub("\[[^\]]*\]", " ", content) #gets rid of everything in brackets
                content = re.sub("\s\s+", " ", content) #maintains single spaced uniformity in content
                content = re.sub(r'[^A-Za-z0-9 .]+', '', content)
                content = re.sub("\ss\s", " ", content)#removes the s that comes after dates
                content = re.sub("\ss+\.+\s", " ", content)#removes the s. that comes after dates
                cleaned_text = re.sub("\s\s+", " ", content) #maintains single spaced uniformity in content


                # Count words after applying the regex
                word_count_after = len(re.findall(r'\w+', cleaned_text))

                # Print the word counts
                print(f"File: {source_file_path}")
                print(f"Word count before: {word_count_before}")
                print(f"Word count after: {word_count_after}\n")
                if word_count_after <= 0.5 * word_count_before:
                    new_entry = {'File:  ': source_file_path, 'Word_Count_Before:  ': word_count_before, 'Word_Count_After:  ': word_count_after}
                    outliers.append(new_entry)
                # Save the processed content to the target file
                with open(target_file_path, 'w', encoding='utf-8') as target_file:
                    target_file.write(cleaned_text)
                file_count += 1

    
    if outliers:
        print('=========================================')
        print('POTENTIAL ERROR WARNING')
        print('There are ' + str(len(outliers)) + ' truncated files out of ' +str(file_count))
        print('here is a list of the files where half or more of text was cut after reference page removal..\n\n')
    

        for item in outliers:
            for key in item:
                print(key,item[key])
            print('-' * 20)
    else:
        print('=========================================')
        print('NO ERRORS DETECTED')

In [ ]:
clean_txt_files("no_references", "cleaned_text")

#LDA CORPUS CREATION
*remove* stop words from documents, and append them to a list

In [ ]:
def txt_to_list(root_dir):
    documents = []
    
    # Walk through the root directory and its subdirectories
    for dir_path, _, _ in os.walk(root_dir):
        # Find all .txt files in the current directory
        txt_files = glob.glob(os.path.join(dir_path, "*.txt"))
        
        # Read the content of each .txt file and append it to the 'documents' list
        for txt_file in txt_files:
            with open(txt_file, 'r', encoding='utf-8') as file:
                content = file.read()
                stc_content = remove_stopwords(content)
                documents.append(stc_content)
                
    return documents

#Basic Lda Model
this was trained on the whole corpus (all the years).
We will train on each year seperately in the near future.

In [ ]:
import gensim
import glob
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import remove_stopwords

# create a list of documents
documents = txt_to_list("cleaned_text")


# tokenize the documents and create a dictionary
tokenized_docs = [doc.split() for doc in documents]
dictionary = corpora.Dictionary(tokenized_docs)

# create a corpus from the documents
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# train the LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=20,
    passes=10,
    random_state=42
)

# print the topics
for topic_id, topic in lda_model.show_topics(num_topics=20):
    print(f"Topic {topic_id}: {topic}")

#CREATE CORPUS FOR BERTOPIC MODEL

In [ ]:
"""
create_corpus creates the corpus in the form of a dictionary of lists of strings. 
It takes tje path to the root directory, then goes down two levels and creates lists
for each year to which it appends all text documents that correspond to that ear.
It tokenizes each text file into sentences.


Parameters:
root_dir (str): path of root directory

Returns:
corpus (dict): a dictionary of lists of strings
"""

def create_corpus(root_dir):
    text_files_dict = {}

    line_count = 0
    # Check if the root directory exists
    if not os.path.isdir(root_dir):
        print(f"{root_dir} is not a valid directory.")
        return text_files_dict

    # Iterate through the directories in the root directory 
    for first_level_subdir in os.listdir(root_dir):
        first_level_subdir_path = os.path.join(root_dir, first_level_subdir)

        # Check if the current entry is a directory
        if os.path.isdir(first_level_subdir_path):
            # Iterate through the second level subdirectories
            for second_level_subdir in os.listdir(first_level_subdir_path):
                second_level_subdir_path = os.path.join(first_level_subdir_path, second_level_subdir)
                # Check if the current entry is a directory and has a numeric title
                if os.path.isdir(second_level_subdir_path) and re.match(r'^\d+$', second_level_subdir):
                    # Create a new list for the current numeric title
                    text_files_dict[second_level_subdir] = []

                    # Iterate through the files in the second level subdirectory
                    for file in os.listdir(second_level_subdir_path):
                          with open(second_level_subdir_path + '/' + file) as f:
                              text = f.read()
                              split_text = text.split(".")

                              for s in split_text:
                                  text_files_dict[second_level_subdir].append(s)
                                  line_count += 1

    print(line_count)
    return text_files_dict

In [ ]:
corpus = create_corpus("cleaned_text")

In [ ]:
print(corpus.keys())

dict_keys(['2019', '2009', '1989', '1999', '1979'])


#Bertopic Model

In [ ]:
"""
run_bertopic
run_bertopic takes the whole corpus as its argument, then
trains a bertopic model on each year dataset, creating text documents with
the chosen number of topics, as well as html file visualizations of the topics
as well. It then saves the model for each yeat.
Parameters:
corpus (dict): the corpus in dict of lists of strings form
topic_num (string): number of topics to be 

Returns:
None
"""

def run_bertopic(corpus, topic_num):
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    topic_model = BERTopic(embedding_model=sentence_model, ctfidf_model=ctfidf_model, verbose=True)#, nr_topics = 200)#, umap_model= umap_model ) #, hdbscan_model=hdbscan_model, calculate_probabilities=True, min_topic_size=10)
    
    for key, year in corpus.items():

        #random.shuffle(year)
        #umap_model = UMAP(n_neighbors=20) #, n_components=10, metric='cosine', low_memory=False)

        #hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean',
        #cluster_selection_method='eom', prediction_data=True, min_samples=2)
        topics = topic_model.fit_transform(year)

        vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))
        topic_model.update_topics(year, vectorizer_model=vectorizer_model)
        topic_model.get_topic_info()
        topic_model.save(key, save_embedding_model=True)
        with open('Bertopic_topics_'+ key +'.txt', 'w') as f:
            f.write("Topics for " + key + "\n")
            figure = topic_model.visualize_barchart(top_n_topics=10)
            figure.write_html("Bertopic_topic_visualization_" + key +".html")
            for topic in range(topic_num):
                topic_info = topic_model.get_topic(topic)
                f.write(f"Topic: {topic}\n")
                f.write("Topic Information: \n")
                f.write(f"{topic_info}\n")


In [ ]:
run_bertopic(corpus, 10)

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

2023-05-01 02:22:20,173 - BERTopic - Transformed documents to Embeddings
2023-05-01 02:22:35,046 - BERTopic - Reduced dimensionality
2023-05-01 02:22:35,294 - BERTopic - Clustered reduced embeddings
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Batches:   0%|          | 0/210 [00:00<?, ?it/s]

2023-05-01 02:25:28,136 - BERTopic - Transformed documents to Embeddings
2023-05-01 02:25:43,214 - BERTopic - Reduced dimensionality
2023-05-01 02:25:43,485 - BERTopic - Clustered reduced embeddings
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



Batches:   0%|          | 0/265 [00:00<?, ?it/s]

2023-05-01 02:28:28,382 - BERTopic - Transformed documents to Embeddings
2023-05-01 02:28:52,714 - BERTopic - Reduced dimensionality
2023-05-01 02:28:53,072 - BERTopic - Clustered reduced embeddings
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



Batches:   0%|          | 0/271 [00:00<?, ?it/s]

2023-05-01 02:32:00,107 - BERTopic - Transformed documents to Embeddings
2023-05-01 02:32:18,943 - BERTopic - Reduced dimensionality
2023-05-01 02:32:19,538 - BERTopic - Clustered reduced embeddings
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



Batches:   0%|          | 0/202 [00:00<?, ?it/s]

2023-05-01 02:34:52,943 - BERTopic - Transformed documents to Embeddings
2023-05-01 02:35:07,948 - BERTopic - Reduced dimensionality
2023-05-01 02:35:08,223 - BERTopic - Clustered reduced embeddings
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



#Top2Vec 
still fleshing out the functionality


In [ ]:
"""
run_top2vec

takes the whole corpus as its argument, then
trains a top2vec model on each year dataset, creating text documents with
the chosen number of topics.

Parameters:
corpus (dict): the corpus in dict of lists of strings form

Returns:
None
"""

def run_top2vec(corpus):   
    for key, year in corpus.items():

        model = Top2Vec(year, embedding_model='universal-sentence-encoder', split_documents =True, document_chunker='sequential', chunk_length=360)
        num_topics = model.get_num_topics()
        with open('top2vec_topic_info'+ key +'.txt', 'w') as f:
            f.write("Topics for " + key + "\n")
            for topic in range(num_topics):
                topic_info = model.get_topics(topic)
                f.write(f"Topic: {topic}\n")
                f.write("Topic Information: \n")
                f.write(f"{topic_info}\n")

In [ ]:
run_top2vec(corpus)

In [157]:
import shutil
from google.colab import files

shutil.make_archive('cleaned_text', 'zip', 'zipped')

'/content/cleaned_text.zip'